# LSTM. 8 featerow - mag, x, y, depth, dist, distance, plate, days_diff

In [15]:
import numpy as np
import tensorflow as tf
import pandas as pd
SEED = 1337
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [16]:
ds_train = tf.data.Dataset.load("../data/ds_train/")
ds_val = tf.data.Dataset.load("../data/ds_val/")
ds_test = tf.data.Dataset.load("../data/ds_test/")

In [17]:
ds_train.cardinality().numpy(), ds_val.cardinality().numpy(), ds_test.cardinality().numpy()

(3294436, 465661, 101326)

In [18]:
BATCH_SIZE = 1024
BLOCK_SIZE = 64

In [19]:
def fix_x_and_y(features, context, labels):
    return features, labels

In [20]:
ds_train = ds_train.map(fix_x_and_y)
ds_val = ds_val.map(fix_x_and_y)
ds_test = ds_test.map(fix_x_and_y)

In [21]:
for x, y in ds_train.batch(BATCH_SIZE).take(1):
    print(x.shape)
    print(y.shape)

(1024, 64, 9)
(1024, 1)


In [22]:
# ys = []
# for x, y in ds_train.batch(BATCH_SIZE):
#     ys.append(y.numpy())
# ys = np.concatenate(ys)
# np.bincount(tf.cast(ys, tf.int32).numpy().flatten())

In [23]:
#neg, pos = np.bincount(tf.cast(ys, tf.int32).numpy().flatten())
neg, pos = 3094591, 199845
total = neg + pos
total, pos, neg

(3294436, 199845, 3094591)

In [24]:
ds_train = ds_train.batch(BATCH_SIZE).shuffle(2000, seed=SEED).prefetch(tf.data.experimental.AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [25]:
@tf.keras.saving.register_keras_serializable()
class MyModel(tf.keras.Model):
    def __init__(self, n_embed):
        super(MyModel, self).__init__()
        self.embed_plate = tf.keras.layers.Embedding(64, n_embed // 9)
        self.embed_dd = tf.keras.layers.Embedding(20, n_embed // 9)
        self.embed_magtype = tf.keras.layers.Embedding(20, n_embed // 9)
        self.embed = tf.keras.layers.Dense(n_embed // 9 * 6)
        self.lstm1 = tf.keras.layers.LSTM(n_embed, return_sequences=True)
        self.lstm2 = tf.keras.layers.LSTM(n_embed)
        self.dense1 = tf.keras.layers.Dense(n_embed, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')
        self.conc = tf.keras.layers.Concatenate(axis=-1)

    def call(self, x):
        cont, plate, dd, magtype = x[:,:,:-3], x[:,:,-3], x[:,:,-2], x[:,:,-1] 
        x1 = self.embed(cont)
        x2 = self.embed_plate(plate)
        x3 = self.embed_dd(dd)
        x4 = self.embed_magtype(magtype)
        x = self.conc([x1, x2, x3, x4])
        x = self.lstm1(x)
        x = self.lstm2(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

In [26]:
model = MyModel(135)

In [27]:
model.build(input_shape=(None, BLOCK_SIZE, 9))
model.count_params()

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
            optimizer='adam',
            metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

In [ ]:
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}
history = model.fit(ds_train, epochs=20, validation_data=ds_val, class_weight=class_weight)

Epoch 1/20


2024-01-09 16:25:10.091888: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 91 of 2000
2024-01-09 16:25:30.102669: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 196 of 2000
2024-01-09 16:25:40.107787: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 260 of 2000
2024-01-09 16:25:50.166723: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 321 of 2000
2024-01-09 16:26:10.345374: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 449 of 2000
2024-01-09 16:26:30.196667: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a w

3218/3218 [==============================] - 1035s 150ms/step - loss: 0.4644 - accuracy: 0.8320 - recall: 0.7507 - precision: 0.2296 - val_loss: 0.4043 - val_accuracy: 0.8843 - val_recall: 0.3701 - val_precision: 0.2317
Epoch 2/20


2024-01-09 16:42:20.007268: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 88 of 2000
2024-01-09 16:42:30.053119: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 176 of 2000
2024-01-09 16:42:40.074982: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 279 of 2000
2024-01-09 16:42:59.997191: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 464 of 2000
2024-01-09 16:43:10.024290: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 597 of 2000
2024-01-09 16:43:20.025818: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a w

3218/3218 [==============================] - 677s 152ms/step - loss: 0.4345 - accuracy: 0.8580 - recall: 0.7488 - precision: 0.2638 - val_loss: 0.4202 - val_accuracy: 0.8639 - val_recall: 0.4157 - val_precision: 0.2061
Epoch 3/20


...

3218/3218 [==============================] - 774s 156ms/step - loss: 0.2227 - accuracy: 0.8969 - recall: 0.9175 - precision: 0.3619 - val_loss: 0.3812 - val_accuracy: 0.8578 - val_recall: 0.5491 - val_precision: 0.2305
Epoch 19/20


2024-01-09 20:11:34.947479: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 70 of 2000
2024-01-09 20:11:54.911166: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 210 of 2000
2024-01-09 20:12:04.914115: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 282 of 2000
2024-01-09 20:12:24.943496: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 420 of 2000
2024-01-09 20:12:44.907991: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 563 of 2000
2024-01-09 20:13:04.850142: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a w

3218/3218 [==============================] - 755s 150ms/step - loss: 0.2062 - accuracy: 0.9028 - recall: 0.9264 - precision: 0.3772 - val_loss: 0.4777 - val_accuracy: 0.8685 - val_recall: 0.5024 - val_precision: 0.2374
Epoch 20/20


2024-01-09 20:24:10.352640: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 86 of 2000
2024-01-09 20:24:30.297024: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 224 of 2000
2024-01-09 20:24:40.308452: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 291 of 2000
2024-01-09 20:24:50.342788: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 369 of 2000
2024-01-09 20:25:00.378988: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 452 of 2000
2024-01-09 20:25:20.331729: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a w

3218/3218 [==============================] - 751s 152ms/step - loss: 0.2009 - accuracy: 0.9057 - recall: 0.9272 - precision: 0.3850 - val_loss: 0.5050 - val_accuracy: 0.8658 - val_recall: 0.5512 - val_precision: 0.2441


In [ ]:
model.evaluate(ds_test)

2024-01-26 06:43:30.344902: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8904
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /home/majkel/miniconda3/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget


99/99 [==============================] - 11s 60ms/step - loss: 0.2812 - accuracy: 0.8920 - recall: 0.7478 - precision: 0.1414


[0.2811870276927948,
 0.8919823169708252,
 0.7478108406066895,
 0.14142584800720215]

In [ ]:
model.save("../models/model_1_cw.keras")

In [ ]:
# save history
history = pd.DataFrame.from_dict(history.history)
history.to_csv("../models/history_model_v1_cw.csv", index=False)